# Set up notebook

In [ ]:
from pyprojroot import here
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
import seaborn as sns

In [ ]:
data_dir = here() / "data" / "raw"

In [ ]:
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", 20)
pd.set_option("display.max_columns", 30)

In [ ]:
%matplotlib widget

# Load data

## Load Freestyle Libre data file

In [ ]:
libre_path = data_dir / "AnaAndres-Arroyo_glucose_21-8-2023.csv"
libre = pd.read_csv(libre_path, header=1, index_col="Device Timestamp")
libre = libre.dropna(axis="columns", how="all")
libre.index = pd.to_datetime(libre.index, format="mixed", dayfirst=True)
libre.shape

In [ ]:
libre.head()

## Load finger pricking glucose readings

In [ ]:
finger_path = data_dir / "Glucose Readings.xlsx"
finger = pd.read_excel(finger_path, header=[0, 1], index_col=0)
finger = finger.dropna(axis="rows", how="all")
finger.index = pd.to_datetime(finger.index).date
finger.shape

In [ ]:
finger.head(3)

# Plot data

## Freestyle Libre data of all days

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[14, 4])
libre["Historic Glucose mmol/L"].dropna().plot(
    ax=ax,
    alpha=0.5,
    color="royalblue",
)
libre["Historic Glucose mmol/L"].dropna().rolling(window=100, center=True).mean().plot(
    ax=ax,
    alpha=0.8,
    color="navy",
    label="rolling mean every 100 points",
)
libre["Scan Glucose mmol/L"].dropna().plot(
    ax=ax,
    marker=".",
    linestyle="",
    alpha=0.1,
    color="navy",
)
ax.axhline(
    y=7.8,
    label="threshold = 7.8 mmol/L",
    color="black",
    alpha=0.5,
    linestyle="--",
)
ax.axhline(
    y=5.3,
    label="threshold = 5.3 mmol/L",
    color="black",
    alpha=0.5,
    linestyle=":",
)
ax.axvline(
    x=pd.to_datetime("16/08/2023 15:00", dayfirst=True),
    label="new sensor",
    color="firebrick",
    alpha=0.5,
    linestyle="--",
)
ax.legend(fontsize=8, bbox_to_anchor=[1, 1])
ax.grid(color="black", alpha=0.2)
ax.set_ylabel("Glucose (mmol/L)")
ax.set_xlabel("")
ax.set_title("Data from Freestyle Libre sensor")
fig.tight_layout()

## Finger pricking data of all days

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[14, 4])
for x in ["Before breakfast", "After breakfast", "After lunch", "After dinner"]:
    df = finger[x].drop(columns=["Comments"]).dropna(how="any")
    df.index = pd.to_datetime(df.index.astype(str) + " " + df["Time"].astype(str))
    df["Glucose (mmol/L) in finger blood"].dropna().plot(
        ax=ax,
        alpha=0.6,
        marker=".",
        markersize=14,
        linestyle="-",
        label=x,
    )
ax.axhline(
    y=7.8,
    label="threshold = 7.8 mmol/L",
    color="black",
    alpha=0.5,
    linestyle="--",
)
ax.axhline(
    y=5.3,
    label="threshold = 5.3 mmol/L",
    color="black",
    alpha=0.5,
    linestyle=":",
)
ax.legend(fontsize=8, bbox_to_anchor=[1, 1])
ax.grid(color="black", alpha=0.2)
ax.set_ylabel("Glucose (mmol/L)")
ax.set_xlabel("")
ax.set_title("Data from finger pricking")
fig.tight_layout()

In [ ]:
fig, axes = plt.subplots(4, 1, figsize=[6, 8], sharex=True, sharey=True)
for ax, x in zip(
    axes, ["Before breakfast", "After breakfast", "After lunch", "After dinner"]
):
    sns.histplot(
        ax=ax,
        data=finger,
        x=(x, "Glucose (mmol/L) in finger blood"),
        kde=True,
        bins=np.arange(3, 8, 0.5),
        alpha=0.1,
        label=x,
    )

axes[0].set_title("Data from finger pricking")
axes[0].axvline(
    x=5.3,
    label="threshold = 5.3 mmol/L",
    color="black",
    alpha=0.5,
    linestyle=":",
)

for ax in axes[1:]:
    ax.axvline(
        x=7.8,
        label="threshold = 7.8 mmol/L",
        color="black",
        alpha=0.5,
        linestyle="--",
    )
for ax in axes:
    ax.legend(fontsize=8, loc="upper left")
    ax.grid(color="black", alpha=0.2)
    ax.set_xlabel("Glucose (mmol/L)")

fig.tight_layout()

## Daily Libre and finger pricking data

In [ ]:
all_dates = pd.Series(libre.index.date).sort_values().unique()
ncols = 5
nrows = int(np.ceil((len(all_dates) / ncols)))
fig, axes = plt.subplots(
    nrows,
    ncols,
    figsize=[ncols * 4, nrows * 2],
    squeeze=False,
    sharex=True,
    sharey=True,
)
for d, ax in zip(all_dates, axes.flatten()[-len(all_dates) :]):
    mask = libre.index.date == d
    df = libre[mask]
    df.index = df.index.time

    # Add glucose readings from the Libre sensor
    df["Historic Glucose mmol/L"].dropna().plot(
        ax=ax,
        alpha=0.8,
        color="royalblue",
    )
    df["Scan Glucose mmol/L"].dropna().plot(
        ax=ax,
        marker=".",
        markersize=6,
        linestyle="",
        alpha=0.1,
        color="navy",
    )

    # Add notes, which are usually meals and snacks
    for t in df["Notes"].dropna().index:
        ax.axvline(
            x=t,
            color="chocolate",
            alpha=0.3,
        )

    # Add thresholds
    ax.axhline(
        y=7.8,
        label="threshold = 7.8 mmol/L",
        color="black",
        alpha=0.5,
        linestyle="--",
    )
    ax.axhline(
        y=5.3,
        label="threshold = 5.3 mmol/L",
        color="black",
        alpha=0.2,
        linestyle="--",
    )

    # Add manual data from finger pricks
    if d in finger.index:
        # Glucose readings
        s = (
            finger.loc[
                d,
                [
                    ("Before breakfast", "Time"),
                    ("Before breakfast", "Glucose (mmol/L) in finger blood"),
                    ("After breakfast", "Time"),
                    ("After breakfast", "Glucose (mmol/L) in finger blood"),
                    ("After lunch", "Time"),
                    ("After lunch", "Glucose (mmol/L) in finger blood"),
                    ("After dinner", "Time"),
                    ("After dinner", "Glucose (mmol/L) in finger blood"),
                ],
            ]
            .unstack()
            .set_index("Time")["Glucose (mmol/L) in finger blood"]
            .dropna()
        )
        if s.size > 0:
            s.plot(
                ax=ax,
                marker=".",
                markersize=8,
                linestyle="",
                alpha=0.8,
                color="firebrick",
            )

        # Meal times
        for m in ["Breakfast", "Lunch", "Dinner"]:
            ax.axvline(
                x=finger.loc[d, (m, "Time")],
                color="chocolate",
                alpha=0.8,
            )

    # Print the date on the plot
    ax.text(0, 8.5, d.strftime("%Y-%m-%d %A"), fontsize=8)

    # Format plot
    ticks = pd.date_range("2023-08-15", "2023-08-16", freq="2H").time[:-1]
    tick_labels = [t.strftime("%H:%M") for t in ticks]
    ax.set_xticks(ticks)
    ax.set_xticklabels(tick_labels, rotation=90)
    ax.xaxis.set_minor_locator(mticker.AutoMinorLocator(2))
    ax.yaxis.set_minor_locator(mticker.AutoMinorLocator(2))
    # ax.legend(fontsize=6, loc='lower left')
    ax.grid(color="black", alpha=0.1, which="major")
    ax.grid(color="black", alpha=0.05, which="minor")
    ax.set_ylabel("Glucose (mmol/L)")
    ax.set_xlabel("Timestamp")
axes.flatten()[-len(all_dates)].legend(fontsize=6, loc="lower left")
axes.flatten()[0].text(
    0,
    7.5,
    "Red dots are finger measurements.\nDark brown lines are meals (breakfast, lunch, dinner).\nFaint brown lines are snacks.",
    fontsize=7,
    va="top",
)
fig.tight_layout()

## Difference between Libre and finger pricks

In [ ]:
df = (
    finger[["Before breakfast", "After breakfast", "After lunch", "After dinner"]]
    .unstack()
    .unstack(1)
)
df.columns.name = ""
df = df.reset_index()
df = df.rename(columns={"level_0": "Reading", "level_1": "Date"})
df = df.set_index(["Date", "Time"])
df["Glucose (mmol/L) difference: finger blood - arm sensor"] = (
    df["Glucose (mmol/L) in finger blood"] - df["Glucose (mmol/L) in arm sensor"]
)
df

fig, ax = plt.subplots(1, 1, figsize=[6, 4], sharex=True)
sns.histplot(
    ax=ax,
    data=df,
    x="Glucose (mmol/L) difference: finger blood - arm sensor",
    kde=True,
    bins=17,
    color="firebrick",
    edgecolor="firebrick",
    alpha=0.1,
)
ax.axvline(0, color="black", linestyle="--", alpha=0.8)
ax.set_ylabel("Number of readings")
ax.grid(color="black", alpha=0.2)
fig.tight_layout()

fig, ax = plt.subplots(1, 1, figsize=[6, 6], sharex=True)
sns.scatterplot(
    ax=ax,
    data=df,
    x="Glucose (mmol/L) in arm sensor",
    y="Glucose (mmol/L) in finger blood",
    hue="Reading",
    # palette='colorblind',
    s=100,
    alpha=0.6,
)
x = ax.get_xlim()
for offset, linestyle in zip([1, 0.5, 0, -0.5, -1], [":", "--", "-", "--", ":"]):
    ax.plot(
        x,
        np.array(x) + offset,
        color="black",
        linestyle=linestyle,
        alpha=0.3,
        label=f"y = x + {offset}",
    )
ax.legend(fontsize=8)
ax.set_aspect("equal")
ax.grid(color="black", alpha=0.2)
fig.tight_layout()

## Time between meals and finger pricks

In [ ]:
times_df = finger.xs("Time", axis="columns", level=1).dropna()
for c in times_df.columns:
    times_df[c] = pd.to_datetime(
        times_df.index.astype(str) + " " + times_df[c].astype(str)
    )
times_df

fig, axes = plt.subplots(3, 1, figsize=[6, 6], sharex=True, sharey=True)
for ax, x in zip(axes, ["Breakfast", "Lunch", "Dinner"]):
    times_df[f"{x} time diff mins"] = (
        times_df[f"After {x.lower()}"] - times_df[x]
    ).dt.total_seconds() / 60
    sns.histplot(
        ax=ax,
        data=times_df,
        x=f"{x} time diff mins",
        kde=True,
        bins=np.arange(45, 95, 5),
        alpha=0.1,
        label=x,
    )
    ax.axvline(
        x=60,
        label="60 minutes",
        color="black",
        alpha=0.5,
        linestyle="--",
    )
    ax.legend(fontsize=8, loc="upper left")
    ax.grid(color="black", alpha=0.2)
    ax.set_xlabel("Time between meal and finger prick (minutes)")
fig.tight_layout()

# TODO

In [ ]:
# TODO: rolling mean to see if the average glucose level is increasing with time over many days
# TODO: compare meals aligned at meal time not time of day
# TODO: calculate sensor peak after each meal